In [ ]:
%%time
import numpy as np
import scanpy as sc
import os
import pandas as pd
import bbknn

sc.settings.verbosity = 4                
sc.settings.set_figure_params(80)                 
sc.settings.file_format_figures = 'png'  
sc.settings.savefigs = False            
use_first_n_samples = 0
full_sparse = False

# Basic QC workflow

In [ ]:
adatas =[]

paths = ['\\adata1','\\adata2','\\adata3'] #...etc - Folders with matrix, features and barcodes gz files.
main_path = r'C:\Users\TzachiHNB5\Documents\scanpy_bat1k\gut' 
for path,name in zip(paths,data_names):
    adata = sc.read_10x_mtx(main_path+ path,var_names='gene_symbols',            # use gene symbols for the variable names (variables-axis index)
        cache=True)  
    sc.logging.print_memory_usage()
    print(adata.shape)
    adata.var_names =genes.index
    adata.var.drop(columns= 'feature_types',inplace=True)
    adata.var.rename(columns= {'gene_ids':'original_name'},inplace=True)
    adatas.append(adata)


In [ ]:
adata_names =['name_1','name_2','name_3'] #...tec
adata = adatas[0].concatenate(adatas[1:], batch_categories = adata_names)

### Genes and cells filtration 


Show those genes that yield the highest fraction of counts in each single cell, across all cells.

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20, )

Basic filtering

In [ ]:
sc.pp.filter_cells(adata, min_genes=200) 
sc.pp.filter_genes(adata, min_cells=3)
print(adata.shape) 

### Mitochondrial QC and general measures

Check if genes are annotated as mt by running:
- GENES= list(adata.var.index[adata.var.index.str.startswith('mt-'.upper())])
- GENES

In case the genes are not annotated as 'MT-'' (Like in bats), run:

- dict_replace = {'COX1':'MT-COX1','COX2':'MT-COX2'...etc}
- adata.var.rename(dict_replace, inplace = True)

In [ ]:
adata.var['MT'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'MT'
sc.pp.calculate_qc_metrics(adata, qc_vars=['MT'], percent_top=None, log1p=False, inplace=True)

A violin plot of some of the computed quality measures:

- the number of genes expressed in the count matrix
- the total counts per cell
- the percentage of counts in mitochondrial genes

In [ ]:
sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts', color='batch')
sc.pl.scatter(adata, x='total_counts', y='pct_counts_MT', color='batch')
sc.pl.scatter(adata, x='total_counts', y='n_genes', color='batch')

Actually do the filtering by slicing the AnnData object - By pct_counts_MT  and by total_counts /n_genes_by_counts or even n_genes

In [ ]:
adata = adata[adata.obs.pct_counts_MT < threshold(int)] 
adata = adata[adata.obs.total_counts < threshold(int), :] # If filtering outliers (<0.1% of cells)

### Doublet analysis and filtering

In [ ]:
import scrublet as scr
def scrub(adatas,adata,adata_names): # based on raw individual samples. 
    import scrublet as scr
    print('Before scrublet: ',adata.shape[0])
    doub_index =[]
    barcodes =[]
    for data,name in zip(adatas,adata_names):
        data.raw = data
        sc.pp.normalize_total(data, target_sum=1e4)
        sc.pp.log1p(data)
        scrub = scr.Scrublet(data.raw.X)
        data.obs['doublet_scores'], data.obs['predicted_doublets'] = scrub.scrub_doublets()
        scrub.plot_histogram()
        print('Doublets'+name +' :',data.obs[data.obs['doublet_scores'] >0.25].shape[0])
        barcodes = data.obs[data.obs['doublet_scores'] <0.25].index.to_list()
        for barcode in barcodes:
            doub_index.append(barcode+'-'+name)
    
    adata = adata[adata.obs.index.isin(doub_index)]
    print('After scrublet: ',adata.shape[0])
    return adata

In [ ]:
adata = scrub(adatas,adata,adata_names) #adata_names in the same order as adatas

### Cell cycle scoring

download Cell cycle txt: https://github.com/scverse/scanpy_usage/blob/master/180209_cell_cycle/data/regev_lab_cell_cycle_genes.txt

In [ ]:
cell_cycle_genes = [x.strip() for x in open('regev_lab_cell_cycle_genes.txt')]

s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]
cell_cycle_genes = [x for x in cell_cycle_genes if x in adata.var_names]
s_genes = [x for x in s_genes if x in adata.var_names]
g2m_genes = [x for x in g2m_genes if x in adata.var_names]

In [ ]:
cell_cycle_adata = adata.copy()

sc.pp.normalize_per_cell(cell_cycle_adata, counts_per_cell_after=1e4)
sc.pp.log1p(cell_cycle_adata)
sc.pp.scale(cell_cycle_adata)
sc.tl.score_genes_cell_cycle(cell_cycle_adata, s_genes=s_genes, g2m_genes=g2m_genes)
adata_cc_genes = cell_cycle_adata[:, cell_cycle_genes].copy()
sc.tl.pca(adata_cc_genes)
sc.pl.pca_scatter(adata_cc_genes, color='phase')
adata.obs['S_score'] = cell_cycle_adata.obs['S_score'].copy()
adata.obs['G2M_score'] = cell_cycle_adata.obs['G2M_score'].copy()
adata.obs['phase'] = cell_cycle_adata.obs['phase'].copy()

### Saving adata

In [2]:
adata.write('file_name.h5ad')